In [24]:
import math as m
import pandas as pn
import numpy as np
import random


### _Helper Classes_

In [41]:
class Graph(object):
    def __init__(self, cost_matrix: list, rank: int):
        """
        :param cost_matrix:
        :param rank: rank of the cost matrix
        
        """
        self.matrix = cost_matrix
        self.rank = rank
        self.pheromone = [[1/rank for j in range(rank)] for i in range(rank)]
        
        
class _Ant(object):
    
    def __init__(self, aco: ACO, graph: Graph, start):
        
        self.colony = aco
        self.graph = graph
        self.total_cost = 0.0
        
        # tabu = The path travelled by the ant - list
        self.tabu = [] 

        # pheromone_delta = the local increase of pheromone
        self.pheromone_delta = []  
        
        # allowed = nodes which are allowed for the next selection - already visited are excluded
        self.allowed = [i for i in range(graph.rank)]  
        
        #eta =  heuristic information
        self.eta = [[0 if i == j else (graph.matrix[i][j] * 0.1) for j in range(graph.rank)] for i in range(graph.rank)]  
        
        #start node
        self.start = start  
        
        #Start node insertion
        self.tabu.append(start)
        self.current = start
        self.allowed.remove(start)

        
        
    def _select_next(self):
        denominator = 0   # for normalization purpose
        
        # The costs (self.graph.matrix[][]) are added with probability to add weight to the nodes with more cost
        for i in self.allowed:
            denominator += self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][i] \
                                                    ** self.colony.beta * self.graph.matrix[self.current][i] ** 2
            
        
        # probabilities for moving to a node in the next step
        probabilities = [0 for i in range(self.graph.rank)]  
        
        for i in range(self.graph.rank):
            
            try:
                self.allowed.index(i)  # test if allowed list contains i
                
                
                # The costs (self.graph.matrix[][]) are added with probability to add weight to the nodes with more cost
                probabilities[i] = self.graph.pheromone[self.current][i] ** self.colony.alpha * \
                    self.eta[self.current][i] ** self.colony.beta * self.graph.matrix[self.current][i] ** 2 / denominator
            
            
            except ValueError:
                pass  # do nothing
            
            
        # select next node by probability roulette. 
        selected = 0
        
        rand = random.random()
        for i, probability in enumerate(probabilities):
            rand -= probability
            
            if rand <= 0:
                selected = i
                break
        
        
        self.allowed.remove(selected)
        self.tabu.append(selected)
        self.total_cost += self.graph.matrix[self.current][selected]
        self.current = selected
        return self.graph.matrix[self.current][selected]

    
    
    def _update_pheromone_delta(self):
        
        self.pheromone_delta = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        
        for _ in range(1, len(self.tabu)):
            i = self.tabu[_ - 1]
            j = self.tabu[_]
            
            if self.colony.update_strategy == 1:  # ant-quality system
                self.pheromone_delta[i][j] = self.colony.Q
            
######## ---------------->  ant-density system -
######## ---------------->
######## ---------------->  Modified : self.pheromone_delta[i][j] = self.colony.Q * self.graph.matrix[i][j] ** 2
######## ---------------->  Before modification : self.pheromone_delta[i][j] = self.colony.Q / self.graph.matrix[i][j]
######## ---------------->
######## ---------------->  Gave better results, but might be the cause of the problem.


            elif self.colony.update_strategy == 2:  
                self.pheromone_delta[i][j] = self.colony.Q * self.graph.matrix[i][j] ** 2
            
            else:  # ant-cycle system
                self.pheromone_delta[i][j] = self.colony.Q / self.total_cost
                
    
    
    #### ------> Added so that the best path found by an ant has more pheromone. The next ants can then recognize the better parts
    def _remember_best_path(self, best_path_till_now):
        
        for _ in range (1, len(best_path_till_now)):
            i = best_path_till_now[_ - 1]
            j = best_path_till_now[_]
            
            #graph.matrix[][] is multiplied so that the nodes with more weight has more pheromone
            self.pheromone_delta[i][j] += self.colony.Q * (self.graph.matrix[i][j] ** (1))   
        
        


### ACO

In [42]:
class ACO(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int,
                 strategy: int):
        """
        :param ant_count:
        :param generations:
        :param alpha: relative importance of pheromone
        :param beta: relative importance of heuristic information
        :param rho: pheromone residual coefficient
        :param q: pheromone intensity
        :param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
        
        """
        self.Q = q
        self.rho = rho
        self.beta = beta
        self.alpha = alpha
        self.ant_count = ant_count
        self.generations = generations
        self.update_strategy = strategy

    def _update_pheromone(self, graph: Graph, ants: list):
        
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                
                graph.pheromone[i][j] *= self.rho
                
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]

    
    def solve(self, graph: Graph):
        """
        :param graph:
        
        """
        best_cost = float('-inf')
        best_solution = []
        
        for gen in range(self.generations):
            
            ants = []
            ant = _Ant(self, graph, random.randint(0, graph.rank - 1))
            ants.append(ant)
            
            best = [ant.graph.matrix[ant.start][0], ant.start]
            
            for loop in range(self.ant_count):
                for i in range(graph.rank - 1):
                    cst = ant._select_next()
                    if cst > best[0]:
                        best = [cst, ant.current]
                

                ant.total_cost += graph.matrix[ant.tabu[-1]][ant.tabu[0]]
                if ant.total_cost > best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.tabu
                    
                    
                # update pheromone
                ant._update_pheromone_delta()
                ant._remember_best_path(best_solution)
                
                if loop != (self.ant_count - 1):
                    ant = _Ant(self, graph,  random.randint(0, graph.rank - 1))
                    ants.append(ant)
            
            
            self._update_pheromone(graph, ants)
            
        return best_solution, best_cost



        

### DNA Fragment Assembly Problem using ACO

In [43]:
from plot import plot

def main():
    
    cnt = 0
    row = []
    
    with open('./data/x60189_4.csv') as f:
        
        for line in f.readlines():
            cnt = cnt + 1
            if (cnt > 2):
                temp = line.split(',')
                n = len(temp)
                
                temp [n - 1] = temp [n - 1].replace("\n", "")
                temp = list(map(int, temp))

                row.append(temp)
        
    cost_matrix = row
    
    rank = (len(row))
   
    ant_number = 100
    genarations = 5
    alpha = 10
    beta = 0
    rho = 0.5          
    q = 0.1            
    strategy = 2       # pheromone update strategy 
    
    aco = ACO(ant_number, genarations, alpha, beta, rho, q, strategy)
    
    
    graph = Graph(cost_matrix, rank)
    path, cost = aco.solve(graph)
    
    print('Cost: {}\nPath: {}'.format(cost, path))
    #plot(points, path)



In [44]:
#Maximum
import time

start = time.time()

if __name__ == '__main__':
    main()
    
end = time.time()

print ("Time taken: " + str(end - start))



Cost: 11481.0
Path: [1, 30, 21, 7, 8, 35, 32, 9, 33, 0, 3, 10, 25, 12, 5, 18, 16, 15, 13, 27, 20, 14, 6, 26, 22, 17, 4, 37, 31, 2, 23, 34, 11, 28, 29, 38, 24, 19, 36]
Time taken: 3.598181962966919
